In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns;
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn import *
from lightgbm import LGBMClassifier
from sklearn.linear_model import *
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
#Load all the data files

train= pd.read_csv('/content/Train.csv')
patient = pd.read_csv('/content/Patient_Profile.csv')
first=pd.read_csv('/content/First_Health_Camp_Attended.csv')
second=pd.read_csv('/content/Second_Health_Camp_Attended.csv')
third=pd.read_csv('/content/Third_Health_Camp_Attended.csv')
test=pd.read_csv('/content/test_l0Auv8Q.csv')
health=pd.read_csv('/content/Health_Camp_Detail.csv')
patient=pd.read_csv('/content/Patient_Profile.csv')

In [ ]:
#EDA 


train.describe()
test['Health_Camp_ID'].nunique()
total=first.shape[0] + second.shape[0] + third.shape[0]
total
train['Health_Camp_ID'].unique()

In [ ]:
#Left joins on all the csv files to get train2(final training file)

train1=pd.merge(train,first,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
train1=pd.merge(train1,second,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
train1=pd.merge(train1,third,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
train2=pd.merge(train1,health,how='left',left_on=['Health_Camp_ID'], right_on=['Health_Camp_ID'])
train2=pd.merge(train2,patient,how='left',left_on=['Patient_ID'], right_on=['Patient_ID'])

In [ ]:
#Merging columns from first,second,third to create a new column of attended or not and dropping irrelevant columns

train2['Health_Score']=train2['Health_Score'].fillna(train2['Health Score'])
train2['Health_Score']=train2['Health_Score'].fillna(train2['Number_of_stall_visited'])
train2=train2.drop(['Last_Stall_Visited_Number','First_Interaction','Health Score','Number_of_stall_visited','Last_Stall_Visited_Number','Unnamed: 4','Patient_ID','Health_Camp_ID','Donation','Employer_Category'],axis=1)
train2.loc[train2['Health_Score'].isnull(), 'Health_Score'] = 0
train2.loc[train2['Health_Score']>0, 'Health_Score'] = 1
train2=train2.rename(columns={"Health_Score": "attended"})
train2.replace('None',np.nan,inplace=True)
train2['City_Type'].fillna(train2['City_Type'].mode()[0],inplace=True)
train2 = train2[['Registration_Date', 'Camp_Start_Date', 'Camp_End_Date', 'Category1','Category2','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','Income','Education_Score','Age','City_Type','Var1','Var2','Var3','Var4','Var5','attended']]

In [ ]:
#Feature Engineering for dates

train2['Registration_Date']=pd.to_datetime(train2['Registration_Date'])
train2['Camp_End_Date'] = pd.to_datetime(train2['Camp_End_Date'])
train2['Camp_Start_Date'] = pd.to_datetime(train2['Camp_Start_Date'])
train2['total_days_of_campaign'] = (train2['Camp_End_Date'] - train2['Camp_Start_Date']).dt.days
is_registration_in_between_campaign=[]
for i in range(len(train2)):
  if train2.iloc[i]['Registration_Date'] >= train2.iloc[i]['Camp_Start_Date'] and train2.iloc[i]['Registration_Date'] <= train2.iloc[i]['Camp_End_Date']:
    is_registration_in_between_campaign.append(1)
  else:
    is_registration_in_between_campaign.append(0)
train2['is_registration_in_between_campaign']=is_registration_in_between_campaign
train2=train2.drop(['Registration_Date','Camp_Start_Date','Camp_End_Date','Income','Education_Score','Age'],axis=1)
train2 = train2[['is_registration_in_between_campaign', 'total_days_of_campaign', 'Category1','Category2','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','City_Type','Var1','Var2','Var3','Var4','Var5','attended']]

In [ ]:
#creating dummy variables for c1,c2 and city 

train2 = pd.get_dummies(train2, columns=['Category1','Category2','City_Type'])

In [ ]:
#Label Encoding for categorical variables

for x in train2.columns:
    if train2[x].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train2[x].values))
        train2[x] = lbl.transform(list(train2[x].values))

In [ ]:
train2.dtypes

is_registration_in_between_campaign      int64
total_days_of_campaign                   int64
Category2                                int64
Category3                                int64
Online_Follower                          int64
LinkedIn_Shared                          int64
Twitter_Shared                           int64
Facebook_Shared                          int64
City_Type                                int64
Var1                                     int64
Var2                                     int64
Var3                                     int64
Var4                                     int64
Var5                                     int64
attended                               float64
Category1_First                          uint8
Category1_Second                         uint8
Category1_Third                          uint8
dtype: object

In [ ]:

train2.groupby('attended').sum()['Category3'].plot(figsize=(12,8))
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 26,
        }
plt.xlabel('Attended',fontdict=font)
plt.ylabel('Category3',fontdict=font)

In [ ]:
train2['Online_Follower'].value_counts()
train2.groupby(['attended','City_Type'])['City_Type'].count()

In [ ]:
train2['attended'].corr(train2['Facebook_Shared'])

0.03776528200711334

In [ ]:
sns.heatmap(train2)

In [ ]:
#Preparing X,Y

X=train2[['is_registration_in_between_campaign','total_days_of_campaign','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','Var1','Var2','Var3','Var4','Var5','Category1_First','Category1_Second','Category1_Third','Category2_A','Category2_B','Category2_C','Category2_D','Category2_E','Category2_F','Category2_G','City_Type_A','City_Type_B','City_Type_C','City_Type_D','City_Type_E','City_Type_F','City_Type_G', 'City_Type_H', 'City_Type_I']]
y=train2['attended']
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.001)

In [ ]:
#Preparing X,Y

X=train2[['is_registration_in_between_campaign','total_days_of_campaign','Category2','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','City_Type','Var1','Var2','Var3','Var4','Var5','Category1_First','Category1_Second','Category1_Third']]
y=train2['attended']
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.001)

In [ ]:
X

,is_registration_in_between_campaign,total_days_of_campaign,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Var1,Var2,Var3,Var4,Var5,Category1_First,Category1_Second,Category1_Third,Category2_A,Category2_B,Category2_C,Category2_D,Category2_E,Category2_F,Category2_G,City_Type_A,City_Type_B,City_Type_C,City_Type_D,City_Type_E,City_Type_F,City_Type_G,City_Type_H,City_Type_I
0,1,59,2,0,0,0,0,4,0,0,0,2,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
1,1,59,2,0,0,0,0,45,5,0,0,7,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,1,751,2,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1,17,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,0,4,2,0,0,0,1,15,1,0,0,7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75273,1,189,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
75274,1,74,2,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
75275,0,12,2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
75276,1,15,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
#Feature selection


fs = SelectKBest(score_func=mutual_info_classif, k= 20)
fs.fit(X,y)
#fs.scores_
X_train_fs=fs.transform(X)
#Xtest_fs=fs.transform(Xtest)
#X_test_fs=fs.transform(X_test)
fs.scores_

array([0.00788712, 0.12486546, 0.0087936 , 0.00132534, 0.00382206,
       0.00203087, 0.00309205, 0.00409295, 0.00187841, 0.00172374,
       0.        , 0.00160319, 0.1110816 , 0.03716116, 0.04496514,
       0.01863965, 0.00035946, 0.        , 0.01085847, 0.03355915,
       0.03368894, 0.04645582, 0.        , 0.01169362, 0.00015909,
       0.        , 0.        , 0.00035723, 0.00138063, 0.00229813,
       0.        ])

Using different classifiers and hyperparameter tuning to obtain the best result

In [ ]:
destimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X, y)
selector.support_

In [ ]:
print(selector.ranking_)
print(selector.score(X,y))
#Y_pred=selector.predict(X_test)

In [ ]:
clf = LGBMClassifier(n_estimators=550, learning_rate=0.05, random_state=1, colsample_bytree=0.5, reg_alpha=0, reg_lambda=0)

#clf.fit(X_train_fs, Y_train, eval_set=[(X_test_fs, Y_test)], verbose=50,
 #        eval_metric = 'auc', early_stopping_rounds = 100)

clf.fit(X, y, verbose=50)

#preds = clf.predict_proba(X_test)[:, 1]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=550, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0, reg_lambda=0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train_fs, y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
clf = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-3))
clf.fit(X, y)

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
regr.fit(X, y)

In [ ]:
reg =  LinearRegression().fit(X, y)

In [ ]:
#Preparing Test Data
test=pd.read_csv('/content/test_l0Auv8Q.csv')
test=pd.merge(test,first,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,second,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,third,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,health,how='left',left_on=['Health_Camp_ID'], right_on=['Health_Camp_ID'])
test=pd.merge(test,patient,how='left',left_on=['Patient_ID'], right_on=['Patient_ID'])



test=test.drop(['Last_Stall_Visited_Number','Health Score','Health_Score','Number_of_stall_visited','Last_Stall_Visited_Number','Unnamed: 4','Patient_ID','Health_Camp_ID','Donation','Employer_Category'],axis=1)
test.replace('None',np.nan,inplace=True)
test['City_Type'].fillna(test['City_Type'].mode()[0],inplace=True)

test['Registration_Date']=pd.to_datetime(test['Registration_Date'])
test['Camp_End_Date'] = pd.to_datetime(test['Camp_End_Date'])
test['Camp_Start_Date'] = pd.to_datetime(test['Camp_Start_Date'])
test['total_days_of_campaign'] = (test['Camp_End_Date'] - test['Camp_Start_Date']).dt.days
is_registration_in_between_campaign=[]
for i in range(len(test)):
  if test.iloc[i]['Registration_Date'] >= test.iloc[i]['Camp_Start_Date'] and test.iloc[i]['Registration_Date'] <= test.iloc[i]['Camp_End_Date']:
    is_registration_in_between_campaign.append(1)
  else:
    is_registration_in_between_campaign.append(0)
test['is_registration_in_between_campaign']=is_registration_in_between_campaign

test=test.drop(['Registration_Date','Camp_Start_Date','Camp_End_Date','Income','Education_Score','Age'],axis=1)

test = test[['is_registration_in_between_campaign', 'total_days_of_campaign', 'Category1','Category2','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','City_Type','Var1','Var2','Var3','Var4','Var5']]


test = pd.get_dummies(test, columns=['Category1','Category2','City_Type'])
#Adding category2_B since it is not present in the test data
test['Category2_B']=np.zeros(len(test))
test['Category2_B'] = test['Category2_B'].astype(int)


test=test[['is_registration_in_between_campaign', 'total_days_of_campaign',
       'Category3', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Category1_First', 'Category1_Second', 'Category1_Third', 'Category2_A',
       'Category2_B', 'Category2_C', 'Category2_D', 'Category2_E',
       'Category2_F', 'Category2_G', 'City_Type_A', 'City_Type_B',
       'City_Type_C', 'City_Type_D', 'City_Type_E', 'City_Type_F',
       'City_Type_G', 'City_Type_H', 'City_Type_I']]

In [ ]:
#Preparing Test Data
test=pd.read_csv('/content/test_l0Auv8Q.csv')
test=pd.merge(test,first,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,second,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,third,how='left',left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])
test=pd.merge(test,health,how='left',left_on=['Health_Camp_ID'], right_on=['Health_Camp_ID'])
test=pd.merge(test,patient,how='left',left_on=['Patient_ID'], right_on=['Patient_ID'])



test=test.drop(['Last_Stall_Visited_Number','Health Score','Health_Score','Number_of_stall_visited','Last_Stall_Visited_Number','Unnamed: 4','Patient_ID','Health_Camp_ID','Donation','Employer_Category'],axis=1)
test.replace('None',np.nan,inplace=True)
test['City_Type'].fillna(test['City_Type'].mode()[0],inplace=True)

test['Registration_Date']=pd.to_datetime(test['Registration_Date'])
test['Camp_End_Date'] = pd.to_datetime(test['Camp_End_Date'])
test['Camp_Start_Date'] = pd.to_datetime(test['Camp_Start_Date'])
test['total_days_of_campaign'] = (test['Camp_End_Date'] - test['Camp_Start_Date']).dt.days
is_registration_in_between_campaign=[]
for i in range(len(test)):
  if test.iloc[i]['Registration_Date'] >= test.iloc[i]['Camp_Start_Date'] and test.iloc[i]['Registration_Date'] <= test.iloc[i]['Camp_End_Date']:
    is_registration_in_between_campaign.append(1)
  else:
    is_registration_in_between_campaign.append(0)
test['is_registration_in_between_campaign']=is_registration_in_between_campaign

test=test.drop(['Registration_Date','Camp_Start_Date','Camp_End_Date','Income','Education_Score','Age'],axis=1)

test = test[['is_registration_in_between_campaign', 'total_days_of_campaign', 'Category1','Category2','Category3','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared','City_Type','Var1','Var2','Var3','Var4','Var5']]
test = pd.get_dummies(test, columns=['Category1'])
for x in test.columns:
    if test[x].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test[x].values))
        test[x] = lbl.transform(list(test[x].values))

#Adding category2_B since it is not present in the test data
#test['Category2_B']=np.zeros(len(test))
#test['Category2_B'] = test['Category2_B'].astype(int)


test=test[['is_registration_in_between_campaign', 'total_days_of_campaign','Category2',
       'Category3', 'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'City_Type','Var1', 'Var2', 'Var3', 'Var4', 'Var5',
       'Category1_First', 'Category1_Second', 'Category1_Third']]

In [ ]:
test_fs=fs.transform(test)
test_fs.shape

(35249, 20)

In [ ]:
preds = selector.predict(test)
sample_submission=pd.read_csv('/content/sample_submmission.csv')
sample_submission['Outcome']=preds
sample_submission.to_csv('RFE.csv', index=None)

In [ ]:
test

,is_registration_in_between_campaign,total_days_of_campaign,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Var1,Var2,Var3,Var4,Var5,Category1_First,Category1_Second,Category1_Third,Category2_A,Category2_B,Category2_C,Category2_D,Category2_E,Category2_F,Category2_G,City_Type_A,City_Type_B,City_Type_C,City_Type_D,City_Type_E,City_Type_F,City_Type_G,City_Type_H,City_Type_I
0,0,66,2,0,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,0,5,2,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1,580,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,0,5,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
4,0,3,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35244,1,9,2,0,0,1,1,12,2,0,0,6,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
35245,0,5,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
35246,0,5,2,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
35247,0,5,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
